# Bayesian Classification By Phoneme Frequency
Camille Girard

Jonah Spicher

## Abstract

For this project we would like to recognize the language of a word based on the phonemes in the word. Given a word, input in its phoneme form, and a predefined dataset of phoneme frequency in each language being tested, we will run through all the phonemes in the word and calculate the probability of each language. Combining all the phonemes of the word will return an updated probability that the observed word is in that language. 

Question: Let's say you are in a place where people are speaking five languages: English, Spanish, Italian, Mandarin, Arabic, and Hindi. You overhear someone say a word, based on the sounds in that word, which language are they speaking? 
Given each of these five words, (language, idioma,  语言, لغة, भाषा)

## Introduction
Usualy, it is fairly easy to tell what language someone is speaking just by hearing it, at least assuming you have heard that language spoken before. The information is being conveyed (obviously) by the sounds they are making, so it should be possible to guess the language of a word given the sounds that make that word up. Bayesian classification provides a fairly straightforward solution: figure out how often those phonemes appear in a given language, and the probability that you heard that language followes from Bayes theorem. 

## Code

In [6]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

import math
import numpy as np

from thinkbayes2 import Pmf, Cdf, Suite
import thinkplot

We start with an even prior. This is obviously not a reasonable decision in real life. At the most basic level, you would want to factor in the percentage of people in the world who speak each language, but in reality your prior will skew heavily towards the domnant languages in your area. For this toy example in a mysterious room with only these five (somewhat random) languages, it seems like a reasonable choice, and will at least show how strong of evidence each word offers. 

In [16]:
languages = ['English', 'Spanish', 'Italian', 'Hindi','Arabic', 'Mandarin']
lang_prior = Pmf(languages)
lang_prior.Print()

Arabic 0.16666666666666666
English 0.16666666666666666
Hindi 0.16666666666666666
Italian 0.16666666666666666
Mandarin 0.16666666666666666
Spanish 0.16666666666666666


### Classification code

Next, we need to define our suite and its parameters. For now, a naive Bayesian model seems fine, but given time we will revisit this decision, as this ignores dipthongs, which often are defining features in a language. Here, though, our data is just a series of phonemes, and our hypothesis is a language. The likelihood, then, is just how often the given phoneme appears in our hypothetical language. We have stored these values in a .csv file, so likelihood is just a simple lookup.

In [12]:
import pandas as pd

#df = pd.read_csv('FILE NAME HERE', delimiter='\t')
#df.head()

In [11]:
class Language(Suite):
    
    def Likelihood(self, data, hypo):
        """
        data: string of phonemes (This should maybe be a list, because some phonemes may be more than one character)
        hypo: name of languages
        """
        language = hypo
        like = 1
        for i in data:
            row = df.loc[df['Phonemes']==i].index
            like *= table[language][row]
        return like

## Results
Now, we just have to feed the Suite each word (in IPA) and it should get a decent idea of which language each word belongs to. 

In [17]:
lang_dist = Language(lang_prior)
word = "leɪŋgwij" # English
lang_dist.Update(word)
lang_dist.Print()

NameError: name 'df' is not defined

In [ ]:
lang_dist = Language(lang_prior)
word = "iðjoma" # Spanish
lang_dist.Update(word)
lang_dist.Print()

In [ ]:
lang_dist = Language(lang_prior)
word = "jujen" # Mandarin maybe??
lang_dist.Update(word)
lang_dist.Print()

In [ ]:
lang_dist = Language(lang_prior)
word = "word" #Arabic will go here, once I figure that out
lang_dist.Update(word)
lang_dist.Print()

So it looks like [it worked or it didnt], because feeding each word gave a [high or low] probability that the word belonged to the correct language.